In [138]:
from datasets import load_from_disk

wiki_datasets = load_from_disk("/opt/ml/data/wiki_preprocessed_droped")
wiki_datasets.load_elasticsearch_index("text", host="localhost", port="9200", es_index_name="wikipedia_contexts")

In [139]:
datasets = load_from_disk("/opt/ml/data/train_dataset")

In [140]:
datasets['validation']

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__', 'chunks'],
    num_rows: 240
})

In [141]:
k = 10
score = 0
correct_examples = []
wrong_set = []
for i, data in enumerate(datasets['validation']):
  query = data['question']
  # query = " ".join(map(lambda x: x[0], filter(lambda x: x[1] == 'NNP' or 'NNG', mecab.pos(query))))
  scores, retrieved_examples = wiki_datasets.get_nearest_examples("text", query, k=k)
  if data['context'] in retrieved_examples['text']:
    correct_examples.append(retrieved_examples)
    score += 1
  else: 
    wrong_set.append(data)

In [142]:
score/len(datasets['validation']) * 100

86.66666666666667

In [171]:
from konlpy.tag import Komoran, Okt

komoran = Komoran()
okt = Okt()

In [172]:
stop_words = set()
error_context = []

In [174]:
from tqdm import tqdm

for data in tqdm(wiki_datasets):
  try:
    tag_list = okt.pos(data['text'])
    for tag in tag_list:
      if tag[1] == 'Noun':
        stop_words.add(tag[0])
  except:
    error_context.append(data)

  1%|          | 546/55962 [00:47<1:08:38, 13.45it/s]

In [167]:
nnp_list = list(filter(lambda x: len(x) > 4, nnp_list))

- 기본 엘라스틱 서치 - 노리토크나이저 : top10 - 88.33
- BM25 - b=0.01 : k1=0.1 77.5
- BM25 - b=0.25, k1=1 88.75
- BM25 - b=0.25, k1=1.5 87.5
- BM25 - b=0.1, k1=1 87.5
- DFR 예시 - 87.083
- DFI 기본 - 76.66